In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install pandas datasets transformers torch matplotlib scikit-learn accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from datasets import load_dataset

dataset = load_dataset("inkoziev/incomplete_utterance_restoration")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df = dataset['train'].to_pandas()

# Создание колонки 'label'
df['label'] = (df['short_phrase'] != df['expanded_phrase']).astype(int)

# Выбор нужных колонок
result_df = df[['short_phrase', 'label']]

# Просмотр первых нескольких строк DataFrame
result_df.head()

,short_phrase,label
0,В восьмом,1
1,изучателем,1
2,нет,1
3,"неплохо, вячеслав",1
4,Смертен ли Гиппарх?,0


In [ ]:
label_counts = df['label'].value_counts()
label_counts


1    122198
0      1732
Name: label, dtype: int64

In [ ]:
import pandas as pd

df_train = pd.read_csv('/content/true_tech_full_data.csv', delimiter=';')
# Удаление чисел и точек в начале сроки
df_train['text'] = df_train['text'].str.replace(r'^\d+\.\s*', '', regex=True)
df_train['text'] = df_train['text'].str.replace('"', '')

In [ ]:
df_train

,text,Label
0,Комнатная служба готова помочь 24 часа в сутки.,0
1,Это всё что он задумал.,0
2,Советский военный корабль обнаруживает Гант и ...,0
3,Многим людям может понадобиться помощь в вычис...,0
4,Наиболее важной функцией РКА остается ее служб...,0
...,...,...
5853,переведи 5000 рублей с кредитной карты на счет...,10
5854,Переведи 15K со счета Купить товар в Интернет-...,10
5855,Переведи 10 тысяч рублей с дебетового счета в ...,10
5856,Переvezdi 3k со сберегательного счета в Amazon,10


In [ ]:
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification, RobertaTokenizer
from sklearn.model_selection import train_test_split, StratifiedKFold
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }


df = df_train
texts = df['text'].tolist()
labels = df['Label'].tolist()

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("deepvk/deberta-v1-distill")
#model = AutoModelForSequenceClassification.from_pretrained("deepvk/deberta-v1-distill", num_labels=11)

tokenizer = RobertaTokenizer.from_pretrained("ai-forever/ruRoberta-large")
model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruRoberta-large", num_labels=11)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.

In [ ]:
#torch.cuda.empty_cache()

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    f1 = f1_score(labels, preds, average='weighted')
    return {
        'f1': f1
    }


In [ ]:
kf = StratifiedKFold(n_splits=2)

for train_index, val_index in kf.split(texts, labels):
    model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruRoberta-large", num_labels=11)
    train_texts, val_texts = [texts[i] for i in train_index], [texts[i] for i in val_index]
    train_labels, val_labels = [labels[i] for i in train_index], [labels[i] for i in val_index]

    train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
    val_dataset = CustomDataset(val_texts, val_labels, tokenizer)
    #print(train_dataset)
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=10,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        #warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        evaluation_strategy="epoch"  # оценивать в конце каждой эпохи
    )


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics  # Добавляем функцию вычисления метрик
    )


    trainer.train()
    break

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.159139,0.970241
2,No log,0.124585,0.980854


Epoch,Training Loss,Validation Loss,F1
1,No log,0.159139,0.970241
2,No log,0.124585,0.980854
3,0.211300,0.121464,0.976359


KeyboardInterrupt: 

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("deepvk/deberta-v1-distill", num_labels=11)


train_dataset = CustomDataset(texts, labels, tokenizer)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch"  # оценивать в конце каждой эпохи
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Добавляем функцию вычисления метрик
)


trainer.train()

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at deepvk/deberta-v1-distill and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


ValueError: Trainer: evaluation requires an eval_dataset.

In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer)

In [ ]:
classifier.save_pretrained('/content/drive/MyDrive/Fintech_models/Classification_model_v2.0')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import os

import requests
from transformers import AutoModelForSequenceClassification, TextClassificationPipeline, AutoTokenizer
import torch
import warnings

warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


class SkillClassifier:
    def __init__(self, base_path):
        # self.prepare_model()
        self.path = base_path
        self.classifier = TextClassificationPipeline(model=SkillClassifier.load_model(self),
                                                     tokenizer=SkillClassifier.load_tokenizer(self))

    def load_model(self):
        model = AutoModelForSequenceClassification.from_pretrained(
            self.path#+'model.safetensors'
        )
        return model

    def load_tokenizer(self):
        tokenizer = AutoTokenizer.from_pretrained("deepvk/deberta-v1-distill")
        return tokenizer

    def get_response(self, text: str) -> list:
        label = self.classifier(text)[0]['label']
        return label

cuda:0


In [ ]:
classifier = SkillClassifier('/content/drive/MyDrive/Fintech_models/Classification_model_v1.0/')

tokenizer_config.json:   0%|          | 0.00/487 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
labels = {0: 'ood',
1: 'check_balance',
2: 'sbp',
3: 'card2card',
4: 'self',
5: 'abroad',
6: 'freepayment',
7: 'kvartplata, jkh',
8: 'mobile_phone',
9: 'transport',
10: 'ishop'}

In [ ]:
classifier('хочу оплатить свои услуги жкх')

[{'label': 'LABEL_7', 'score': 0.9902752637863159}]

In [ ]:
classifier.get_response('сколько денег?')

'LABEL_1'

In [ ]:
classifier.get_response('оплати ка мне жкх бот')

'LABEL_0'